<a href="https://colab.research.google.com/github/raneye99/COMP0087/blob/main/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone Git Repo into Google Drive

Note: you will need to create a personal access token and store it into the directory where you will clone the repo.

In [ ]:
#set up
gdrive_location = "/content/drive/MyDrive/NLP"
pat_file = "pat.json"
import subprocess
import json
from google.colab import drive
import os
import sys


gdrive_mount = '/content/drive'
# requires giving access to google drive account
drive.mount(gdrive_mount)

# change to relevant workspace 
# - google drive dir - where 'pat_file' should be found
gdrive_dir = os.path.join(gdrive_mount, gdrive_location)
work_dir = gdrive_dir

#url for repo
url = ":x-oauth-basic@github.com/raneye99/COMP0087"

#repo directory
repo_dir = os.path.join(gdrive_dir, os.path.basename(url))

# change to working directory
# os.chdir(work_dir)
assert os.path.exists(work_dir), f"workspace directory: {work_dir} does not exist"
os.chdir(work_dir)
# this is a bit long - would be good to reduce
# if git directory does not exist - clone 
if not os.path.exists(repo_dir):
    # get pat file - expected to be in work_dir
    # pat expected to be dict: {"pat": "<personal access token>"}
    with open(os.path.join(gdrive_dir, pat_file), "r+") as f:
        pat = json.load(f)

    # git clone
    print(f"cloning directory: {url}")
    git_clone = subprocess.check_output( ["git", "clone", f"https://{pat['pat']}{url}"] , shell=False)
    print(git_clone)

print(f"changing directory to: {repo_dir}")
os.chdir(repo_dir)

# --
# change branch - review this
# --

try:
    branch_name = "main"
    git_checkout = subprocess.check_output(["git", "checkout", "-t", f"origin/{branch_name}"], shell=False)
    print(git_checkout.decode("utf-8") )
except Exception as e:
    git_checkout = subprocess.check_output(["git", "checkout",  f"{branch_name}"], shell=False)
    print(git_checkout.decode("utf-8") )

# ---
# git pull
# ---
# could be redunant if just clone, but whatever

git_pull = subprocess.check_output(["git", "pull"], shell=False)
print(git_pull.decode("utf-8") )


if repo_dir not in sys.path:
    print(f"adding {repo_dir} to sys.path")
    sys.path.extend([repo_dir])


Mounted at /content/drive
changing directory to: /content/drive/MyDrive/NLP/COMP0087
Your branch is up to date with 'origin/main'.

Already up to date.

adding /content/drive/MyDrive/NLP/COMP0087 to sys.path


In [ ]:
#import libraries
import pickle
from collections import defaultdict
from urllib.request import urlopen
import numpy as np
import pandas as pd
import os
import h5py
import sys
import requests
import zipfile
import inspect

Download Data

In [ ]:
#load data

#file located here: https://drive.google.com/file/d/1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P/view

#make data directory if doesn't exist in path folder
os.makedirs('data', exist_ok=True)

#download meld and mosei zip files for data (mosei is from CMU, meld is friends episodes)
if('mosei' in  os.listdir('data')):
    pass
else:
    !file=1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='${file} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="${file} -O data/data.zip && rm -rf /tmp/cookies.txt

    #unzip data
    !unzip data/data.zip -d data/mosei

Preprocess our data

In [ ]:
#preprocess data
from mtl.preprocess import clean_text, create_embedding_matrix, data_processing

X_train, X_test, X_train_pad, X_test_pad, y_train, y_test, y_train1, y_test1, y_train2, y_test2, y_train3, y_test3, y_train4, y_test4, y_train5, y_test5, y_train6, y_test6, embedd_matrix, vocab_size, X_val, X_val_pad, y_val, y_val1, y_val2, y_val3, y_val4, y_val5, y_val6, data_test = data_processing()

y_mtl_train = (y_train, y_train1, y_train2, y_train3, y_train4, y_train5, y_train6 )
y_mtl_val = (y_val, y_val1, y_val2, y_val3, y_val4, y_val5, y_val6)

y_emo_train = (y_train1, y_train2, y_train3, y_train4, y_train5, y_train6 )
y_emo_val = (y_val1, y_val2, y_val3, y_val4, y_val5, y_val6)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Training data sentiment statistics
4.0    4120
3.0    3000
2.0    2040
5.0    1549
1.0    1305
Name: sentiment, dtype: int64
Test data sentiment statistics
3.0    1927
4.0    1230
2.0     554
5.0     441
1.0     399
Name: sentiment, dtype: int64
Validation data sentiment statistics
3.0    835
4.0    470
2.0    244
5.0    169
1.0    110
Name: sentiment, dtype: int64
[ 'website ' 'sp ' 'at ' 'sp ' 'if ' 'you ' 'get ' 'any ' 'other ' 'cool ' 'ideas' 'like ' 'susan ' 'williams ' 'shared ' 'with ' 'me ' 'come ' 'on ' 'and ' 'share' 'them ' 'with ' 'me ' `` i 'll '' 'be ' 'happy ' 'to ' ]
Number of unique words: 18315
{'1.0': 0, '2.0': 1, '3.0': 2, '4.0': 3, '5.0': 4}
Words found in wiki vocab: 653
New words found: 17662


#Expiriments

##Expiriment One: Baseline LSTM with 5 sentiments

Train

In [ ]:
#train

from mtl.network import net
from mtl.train import train_model
from tensorboard.plugins.hparams import api as hp

model_name = 'lstm_one_task_sentiment_five'
saved_model_name = 'models/'+ model_name + '.h5'
print('models/'+ model_name + '.h5')
batch_size=32
epochs=10
gru_output_size=128
dropout=0.2
recurrent_dropout=0.2
tensorboard = True
loss_weights = [0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
print(vocab_size)


train_model(vocab_size = vocab_size, embedd_matrix = embedd_matrix, data_x = X_train_pad, data_y = y_train, val_x = X_val_pad, val_y = y_val, model_name=model_name, saved_model_name=saved_model_name, batch_size=32, epochs=epochs, tensorboard=tensorboard, loss_weights=loss_weights)

models/mtl_five_sentiments20220404-114111.h5
18316
Training a mtl_five_sentiments model!
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 300)     5494800     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 256)          330240      ['embedding[0][0]']              
                                                                                                  
 s (Dense)                      (None, 5)            1285        ['bidirectional[0][0]']          
     

Predict

In [ ]:
#predict
from mtl.predict import test_model

file_name = 'models/lstm_one_task_sentiment_five.h5'
num_classes = 5
num_classes1 = 2
embed_num_dims = 300
max_seq_len = 500

#change those according to the model you are testing
num_of_sentiments = True

test_model(file_name, mtl = False, data_y= data_test, data_x=X_test_pad)